# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

In [ ]:
import os
import os.path
import cv2
import random
import pickle
import numpy as np 
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from itertools import product

---
## Step 0: Load The Data

In [ ]:

def get_files():
    '''
    Helper function to get the pickled data file paths.
    TODO: Get a better way to the data. It might be better to 
          download an unpack the pickeled data.
    '''
    DATA_PATH="/home/carnd/data"
    return (os.path.join(DATA_PATH, "train.p"), 
            os.path.join(DATA_PATH, "test.p"))

def get_csv_file():
    '''
    Helper function to get the traffic sign names CSV file.
    '''
    cwd = os.getcwd()
    filename = os.path.join(cwd, "signnames.csv")
    return filename

def get_test_image_path():
    cwd = os.getcwd()
    path = os.path.join(cwd,"test_images")
    return path

In [ ]:
'''
Helper functions to preprocess images.
'''

def rgb2gray(func):
    def func_wrapper(*args, **kwargs):
        return cv2.cvtColor(func(*args, **kwargs), cv2.COLOR_RGB2GRAY)
    return func_wrapper

def gray2rgb(func):
    def func_wrapper(*args, **kwargs):
        return cv2.cvtColor(func(*args, **kwargs), cv2.COLOR_GRAY2RGB)
    return func_wrapper

def gray2equalizeHist(func):
    def func_wrapper(*args, **kwargs):
        return cv2.equalizeHist(func(*args, **kwargs))
    return func_wrapper
        
def rgb2hsv(func):
    def func_wrapper(*args, **kwargs):
        return cv2.cvtColor(func(*args, **kwargs), cv2.COLOR_RGB2HSV)
    return func_wrapper

def hsv2rgb(func):
    def func_wrapper(*args, **kwargs):
        return cv2.cvtColor(func(*args, **kwargs), cv2.COLOR_HSV2RGB)
    return func_wrapper

def hsv2equalizeHist(func):
    def func_wrapper(*args, **kwargs):
        h,s,v = cv2.split(func(*args, **kwargs))
        v = cv2.equalizeHist(v)
        return cv2.merge((h,s,v))
    return func_wrapper

def rgb2normalize(func):
    def func_wrapper(*args, **kwargs):
        r,g,b = cv2.split(func(*args, **kwargs))
        x = r.copy()
        r = cv2.normalize(r,x)
        g = cv2.normalize(g,x)
        b = cv2.normalize(b,x)        
        return cv2.merge((r,g,b))
    return func_wrapper    
def no_image(image):
    return image

def rgb2randomRotate(func):
    def func_wrapper(*args, **kwargs):
        img = func(*args, **kwargs)
        rotation = random.uniform(-5, 5)
        rotation_x = random.uniform(-5,5)
        rotation_y = random.uniform(-5,5)        
        (rows,cols,ch) = img.shape
        M = cv2.getRotationMatrix2D((cols/2+rotation_x,rows/2+rotation_y),rotation,1)
        dst = cv2.warpAffine(img,M,(cols,rows))
        return dst
    return func_wrapper    
def no_image(image):
    return image

def rgb2randomTranslate(func):
    def func_wrapper(*args, **kwargs):
        img = func(*args, **kwargs)
        (rows,cols,ch) = img.shape
        rotation_x = random.uniform(-5, 5)
        rotation_y = random.uniform(-5, 5 )
        M = np.float32([[1,0,rotation_x],[0,1,rotation_y]])
        dst = cv2.warpAffine(img,M,(cols,rows))        
        return dst
    return func_wrapper    
def no_image(image):
    return image

@rgb2randomTranslate
def translate_image(image):
    return image

@rgb2randomRotate
def rotate_image(image):
    return image

@rgb2normalize
def norm_image(image):
    return image

@gray2rgb
@gray2equalizeHist
@rgb2gray
def gray_image(image):
    return image

@hsv2rgb
@hsv2equalizeHist
@rgb2hsv
def image2(image):
    return image

def translate_images(images):
    X = [ translate_image(features) for features in images]
    Y = X
    return Y

def convert_images(images):
    X = [ no_image(features) for features in images]
    Y = X
    return Y

def create_images(images):
    X = [ rotate_image(features) for features in images]
    Y = X
    return Y

<a id='create_additional_data'>&nbsp;</a>
Load the test and train data and enhance the test data with further generated data.

Goto [implementation](#implementation).

In [ ]:
# Load pickled data
training_file, testing_file = get_files()

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

'''
Create some additional data from the original test data.
Add some rotated and translated images.
'''
train['original_features'] = train['features']
train['features'] = np.concatenate((convert_images(train['features']), 
                                    create_images(train['features']),
                                    create_images(train['features']), 
                                    create_images(train['features']),
                                    translate_images(train['features'])
                                   ))
'''
Ensure that the labels have the same size and contain the right data.
'''
train['labels'] = np.concatenate((train['labels'], 
                                  train['labels'],
                                  train['labels'],
                                  train['labels'],
                                  train['labels']))
test['features'] = convert_images(test['features'])

X_train, X_validation, y_train, y_validation = train_test_split(train['features'], train['labels'], test_size=0.20, random_state=42)
X_test, y_test = test['features'], test['labels']



---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.
n_original_train = len(train['original_features'] )
n_train = len(X_train)
n_validation = len(X_validation)
n_test = len(X_test)
image_shape = [len(X_train[0]),len(X_train[0][0]),len(X_train[0][0][0])]
n_classes = len(set(y_train))

print("Number of loaded training examples =", n_original_train)
print("Number of enhanced training examples minues the extracted validation examples =", n_train)
print("Number of from training examples extracted validation examples =", n_validation)
print("Number of loaded testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
'''
Get the traffic sign names
'''
with open(get_csv_file(), 'rt') as csvfile:
    spamreader = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    signname = [ row['SignName'] for row in spamreader]

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
# Visualizations will be shown in the notebook.
%matplotlib inline

foo = np.unique(y_train,return_counts=True)
count = foo[1]
plt.suptitle("Distribution of traffic signs in train data", fontsize=14, fontweight='bold')
plt.pie(count, labels=signname,shadow=False,startangle=120)
plt.axis('equal')

plt.show()

In [ ]:
# Plot the first occurence of each sign
for i in range(len(signname)):
    a = np.where(y_train==i)[0][0]
    plt.suptitle(signname[i], fontsize=14, fontweight='bold')
    plt.imshow(X_train[a])
    plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [5]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.
pass

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

I've tried several ways to preprocess the data:
* Convert to a grayscale image.
* Convert to a grayscale image and equalize the histogram.
* Convert from an RGB to a HSV (hue, saturation and value) image.
* Equalize the histogram of each channel from an RGB image.
* Convert to a HSV image, equalize the histogram of V and convert it back to an RGB image.

Unfortunately none of these methods improved the accuracy of the validation data. Finally, I decided do not preprocess the data.
For the sake of completeness I've left the conversion implementation in the notebook.

In [6]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.
pass

Already done in the [cell above](#create_additional_data)

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

### Additional data
I created additional data using rotation and translation.
The rotation is applied randomly with a degree of -5 to +5 degree. The center of the rotation is also randomly set -5 to +5 pixels in both directions from the center of the image.

The translation (shift of the image) was applied -5 to +5 pixels in both directions.

I've added more rotated images than translated images to the training set as they improved the accuracy of the validation set.

The creation of the additional data was done in the [cell above](#create_additional_data)

### Set up the data
The validation data was extracted from the loaded and with additional data enhanced test data. 20% of the test data has been used for the validation data.

The testing data is just the data loaded from the pickled test data.

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

EPOCHS = 40
BATCH_SIZE = 64

In [ ]:
def nprint(c):
    """
    print the input tensor and return it
    """
    print(c)
    return c

def LeNet_mod(x):   
    '''
    A modified network based on the LeNet model.
    The input tensor x needs to have a shape of (?, 32, 32, <depth>).
    It will return a graph representing the modified LeNet model
    '''
    (unknown, width, height, depth) = x.get_shape()

    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    weights = {
        'wc1': tf.Variable(tf.truncated_normal([5, 5, int(depth), 6], mean=mu, stddev=sigma)), 
        'wc2': tf.Variable(tf.truncated_normal([1, 1, 6, 6], mean=mu, stddev=sigma)), 
        'wc3': tf.Variable(tf.truncated_normal([1, 1, 6, 6], mean=mu, stddev=sigma)),
        'wc4': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=mu, stddev=sigma)),
        'wfc1': tf.Variable(tf.truncated_normal([400, 120], mean=mu, stddev=sigma)),
        'wfc2': tf.Variable(tf.truncated_normal([120, 84], mean=mu, stddev=sigma)),
        'out': tf.Variable(tf.truncated_normal([84, n_classes], mean=mu, stddev=sigma))
    }


    biases = {
        'bc1': tf.Variable(tf.truncated_normal([6], mean=mu, stddev=sigma)),
        'bc2': tf.Variable(tf.truncated_normal([6], mean=mu, stddev=sigma)),
        'bc3': tf.Variable(tf.truncated_normal([6], mean=mu, stddev=sigma)),
        'bc4': tf.Variable(tf.truncated_normal([16], mean=mu, stddev=sigma)),
        'bfc1': tf.Variable(tf.truncated_normal([120], mean=mu, stddev=sigma)),
        'bfc2': tf.Variable(tf.truncated_normal([84], mean=mu, stddev=sigma)),
        'out': tf.Variable(tf.truncated_normal([n_classes], mean=mu, stddev=sigma))
    }  
    
    ksize = [1, 2, 2 , 1]
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    
    c = x
    print("Layer 1")
    c = nprint(tf.nn.conv2d(c, weights['wc1'], strides=[1, 1, 1, 1], padding='VALID', name="C1.conv2d"))
    c = nprint(tf.nn.bias_add(c, biases['bc1'], name="C1.bias"))
    c = nprint(tf.nn.relu(c, name="C1.relu"))
    
    print("Layer 2")    
    c = nprint(tf.nn.conv2d(c, weights['wc2'], strides=[1, 1, 1, 1], padding='VALID', name="C2.conv2d"))
    c = nprint(tf.nn.bias_add(c, biases['bc2'], name="C2.bias"))
    c = nprint(tf.nn.relu(c, name="C2.relu"))
    
    print("Layer 3")    
    c = nprint(tf.nn.conv2d(c, weights['wc3'], strides=[1, 1, 1, 1], padding='VALID', name="C3.conv2d"))
    c = nprint(tf.nn.bias_add(c, biases['bc3'], name="C3.bias"))
    c = nprint(tf.nn.relu(c, name="C3.relu"))
    
    print("Pooling 1")    
    c = nprint(tf.nn.max_pool(c, ksize, strides, padding, name="P1.max_pool"))
    
    print("Layer 4")    
    c = nprint(tf.nn.conv2d(c, weights['wc4'], strides=[1, 1, 1, 1], padding='VALID', name="C4.conv2d"))
    c = nprint(tf.nn.bias_add(c, biases['bc4'], name="C4.bias"))    
    c = nprint(tf.nn.relu(c, name="C4.relu"))
    
    print("Pooling 2")     
    c = nprint(tf.nn.max_pool(c, ksize, strides, padding, name="P2.max_pool"))

    print("Flatten")        
    c = nprint(flatten(c))

    print("Dropout")        
    c = nprint(tf.nn.dropout(c, keep_prob, name="Dropout"))

    print("Fully Connected 1")    
    c = nprint(tf.add(tf.matmul(c, weights['wfc1']), biases['bfc1'], name="fc1.matmul"))
    c = nprint(tf.nn.relu(c, name="fc1.relu")) 

    print("Fully Connected 2")        
    c = nprint(tf.add(tf.matmul(c, weights['wfc2']), biases['bfc2'], name="fc2.matmul"))
    c = nprint(tf.nn.relu(c, name="fc2.relu"))
    
    print("Fully Connected 3")    
    logits = nprint(tf.add(tf.matmul(c, weights['out']), biases['out'], name="out"))
    return logits

In [ ]:
x = tf.placeholder(tf.float32, (None, image_shape[0], image_shape[1], image_shape[2]), name="x")
y = tf.placeholder(tf.int32, (None), name="y")
# on dropout
keep_prob = tf.placeholder(tf.float32, name="keep_prob")
one_hot_y = tf.one_hot(y, n_classes)

In [3]:
rate = 0.0005

logits = LeNet_mod(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

NameError: name 'LeNet_mod' is not defined

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ... Validation Accuracy = {:.3f}".format(i+1,validation_accuracy))
        
    try:
        saver
    except NameError:
        saver = tf.train.Saver()
    saver.save(sess,'/home/carnd/lenet')
    print("Model saved")

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

![network model](network.svg)

The network architecture is based on LeNet. In addition I took some inspiration from [VGGNet](https://arxiv.org/pdf/1409.1556.pdf) and added a dropout layer.
The network is build 
1. Input layer: Takes a 32x32 Image with an( arbitrary number of channels. In this class the rgb channels have been used.
2. Convolution network 1 (C1): The output shape will be 6@28x28
3. Convolution network 2 (C2): The output shape will be 6@28x28. *Change to original LeNet*.
4. Convolution network 3 (C3): The output shape will be 6@28x28. *Change to original LeNet*.
5. Pooling layer 1 (P1): The output shape is reduced to 6@14x14.
6. Convolution network 4 (C4): The output shape will be 16@7x7. 
7. Pooling layer 2 (P2): The output shape is reduced to 16@5x5.
8. Flattening from 3D to 1D with 400 Features.
9. Dropout layer. *Change from original LeNet*
10. Fully connected Layer 1 (FC1). Reduce to 120 Features.
10. Fully connected Layer 2 (FC2). Reduce to 84 Features.
10. Final output layer 1 (Out). Reduce to final 43 possible traffic sign features.

In the description I left out the RelU and Bias layers from the implementation. They can be seen exactly when calling the `LeNet_mod(x)` function (see above).




In [ ]:
# Finally use the test set to evaluate how good the model was trained.

with tf.Session() as sess:
    loader = tf.train.import_meta_graph('/home/carnd/lenet.meta')
    loader.restore(sess, tf.train.latest_checkpoint('/home/carnd/'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

In [2]:
print("Learning rate: ", rate)
print("Epochs: ", EPOCHS)
print("Batch size: ", BATCH_SIZE)

NameError: name 'rate' is not defined

* Hyperparameters
 * mu = 0
 * sigma = 0.1
* Optimizer: Adam using softmax and one hot encoding

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

As it was suggested I started with the LeNet we've learned in the classroom. In additional I've  looked into other networks (especially VGGNet). After trying VGGNet the accurarcy was not as good as the one from LeNet. So I've tried to mix them both by adding more convolutional layers into the LeNet. While in VGGNet more convolutional layers are added in later steps, it seemed better for this implementation to do this at the beginning of the network. 



---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
dirs = os.listdir(get_test_image_path())
%matplotlib inline
image_batch = []

for mfile in dirs:
    img = cv2.imread(os.path.join(get_test_image_path,mfile))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    (width, heigth, channels) = img.shape
    z=width
    step=(width+heigth)*2
    for (xx,yy) in product(range(0,width,step),range(0,heigth,step)):
        (rows,cols,ch) = img.shape
        rotation = 3
        M = cv2.getRotationMatrix2D((cols/2,rows/2),rotation,1)
        dst = cv2.warpAffine(img,M,(cols,rows))
        #dst = image2(dst)
        small_img = resized_image = cv2.resize(dst[xx:xx+z,yy:yy+z], (32, 32)) 
        image_batch.append(np.copy(small_img))
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    predictions = sess.run(tf.nn.top_k(tf.nn.softmax(logits), k=5), feed_dict={x: image_batch, keep_prob: 1.0})
    #print(predictions)
    
for myfile, value, predicted, num in zip(dirs,predictions[0], predictions[1],range(len(predictions[0]))):
    pass
    #arg= np.argmax(predicted)
    for i in range(len(value)):
        print(value[i], predicted[i], signname[predicted[i]])
    #if predicted[arg]>0.1 :
    plt.suptitle(myfile, fontsize=14, fontweight='bold')
    plt.imshow(image_batch[num])
    plt.show()

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.